# Housing Prices - Evaluation

### James Mwakichako - jmwakich@hawk.iit.edu
### Michael Baroody  - mbaroody@hawk.iit.edu

### Preprocessing

Before we are able to fit our model, we have to take care of missing values and categorical features. 

In [45]:
%matplotlib inline
import pandas as pd
from ipywidgets import widgets
from IPython.display import display
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing, linear_model

# train DataFrame object
# see http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html
# and http://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html
train = pd.read_csv("train.csv", header = 0)

Recall that we had some features with many missing values. Below are all the features that have some missing values. All the other features have all values filled in. 

In [46]:
print("Feature \tProportion Values Missing")
print("------- \t----------------------")

# there are 19 features that contain missing values 
# see http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.count.html#pandas.DataFrame.count
print((1 - (train.count() / len(train))).sort_values(ascending=False).nlargest(19))

Feature 	Proportion Values Missing
------- 	----------------------
PoolQC          0.995205
MiscFeature     0.963014
Alley           0.937671
Fence           0.807534
FireplaceQu     0.472603
LotFrontage     0.177397
GarageCond      0.055479
GarageType      0.055479
GarageYrBlt     0.055479
GarageFinish    0.055479
GarageQual      0.055479
BsmtExposure    0.026027
BsmtFinType2    0.026027
BsmtFinType1    0.025342
BsmtCond        0.025342
BsmtQual        0.025342
MasVnrArea      0.005479
MasVnrType      0.005479
Electrical      0.000685
dtype: float64


We want to throw out all features that have > 25% missing values. That means 'PoolQC', 'MiscFeature', 'Alley', 'Fence', and 'FireplaceQu' will all be excluded from the training data. 

In [47]:
del train['PoolQC']
del train['MiscFeature']
del train['Alley']
del train['Fence']
del train['FireplaceQu']

We will fill in missing values for the rest of the features. For numerical features, we will to fill in the missing values with the mean for that column. For example, we know that the mean value for all of the known 'LotFrontage' values is around 70. Therefore, for all of the 'NaN' values encountered in the 'LotFrontage' column, we will replace the value with 70.

In [48]:
# fill in the missing numerical features with the means
# see http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.mean.html#pandas.DataFrame.mean
# and http://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.fillna.html#pandas.Series.fillna
means = train.mean(skipna=True)
for feature,mean in means.iteritems():
    train[feature] = train[feature].fillna(value=mean)

For the categorical features, we will fill in the missing values with the mode category for that column. For example, it is known that the most frequent 'GarageQual' is 'TA.' Therefore, for all of the 'NaN' values we encounter in the 'GarageQual' column, we will replace the value with 'TA.'

In [49]:
# fill in the missing categorical values with the modes
# see http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.mode.html#pandas.DataFrame.mode
train = train.fillna(train.mode().iloc[0])

Now we must deal with the categorical variables. We do this by encoding the categorical feature labels with numbers, using sklearn.preprocessing. In the visualization phase of our project, we identified 49 categorical features. However, some of these were already encoded with numbers. The 'MoSold' feature, for example, is a categorical feature of the month the house was sold, and it is already encoded for us by the number of the month in the calendar. Therefore, we leave it alone. 

In [50]:
# first, find those features that are unencoded categorical
categorical_features = [feat for feat in train.columns.values if train[feat].dtype == 'object']

# see http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html
le = preprocessing.LabelEncoder()
for feature in categorical_features: 
    le = le.fit(train[feature])
    train[feature] = pd.Series(le.transform(train[feature]))

In [58]:
# see http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html#sklearn-linear-model-lasso
# and http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.drop.html#pandas.DataFrame.drop
clf = linear_model.Lasso(alpha=1.0)
X = train.drop('SalePrice', 1).values
y = train['SalePrice'].values
clf.fit(X, y)

print('Weight coefficients')
print(clf.coef_)

print('\n\nAccuracy score against training data')
print(clf.score(X, y))

Weight coefficients
[ -1.66119746e+00  -1.29602605e+02  -1.24410096e+03  -1.45211548e+02
   3.87104923e-01   2.80078001e+04  -9.22809200e+02   3.45221005e+03
  -4.65932679e+04  -3.99360616e+01   5.81040276e+03   3.72018174e+02
  -7.97868266e+02  -8.52625325e+03  -2.17221654e+03  -1.17298492e+03
   1.10144555e+04   4.57642869e+03   1.84152153e+02   3.59566976e+01
   2.35684078e+03   4.79364387e+03  -9.61174416e+02   3.46186718e+02
   4.69590028e+03   3.14097577e+01  -9.23223168e+03   5.58599930e+02
   1.12790617e+03  -9.07065474e+03   2.45184328e+03  -3.84213815e+03
  -8.96370622e+02   8.88613572e+00   7.43838814e+02   1.11509560e+01
   1.49002350e+00   1.07819462e+00  -1.85145039e+03  -7.48642437e+02
  -6.94094825e+02  -2.82247408e+02   4.25302884e+01   4.19720946e+01
   1.81050397e+01   5.12868229e+00   7.08554153e+03   8.76206687e+02
   3.69514673e+03  -8.88164483e+01  -3.58946330e+03  -1.09788013e+04
  -8.93105230e+03   3.24453047e+03   4.17764582e+03   4.62139890e+03
  -1.23988190e